# Examine features in training and test sets for different datasets

In [18]:
# Load features
import os
import pickle
from tqdm.notebook import tqdm

zero_test_features = {}
total_features = {}
runs = [
    'baseline_randomtest+exp1+exp2_all_lr'
    'dataset114k+exp1+exp2_all_lr',
]

for run in runs:
    print(run)
    run_fname = run + '_features.pkl'
    features_dirpath = '/projects/websci2020_tumblr_identity/output/features'
    features_fpath = os.path.join(features_dirpath, run_fname)
    with open(features_fpath, 'rb') as f:
        X_train, y_train, X_test, y_test = pickle.load(f)
    print(X_train.shape)
    print(X_test.shape)
    print()

    # Count features seen in training set that are never seen in the test set
    X_train_arr = X_train.A
    empty_features_train = []
    for col in tqdm(range(X_train.shape[1])):
        if not X_train_arr[:,col].any():
            empty_features_train.append(col)
    print(len(empty_features_train)) # why are there any?
    total_features[run] = len(X_train.shape[1])
    print(X_train.shape[1])
    print()

    X_test_arr = X_test.A
    empty_features_test = []
    for col in tqdm(range(X_test.shape[1])):
        if not X_test_arr[:,col].any():
            empty_features_test.append(col)
    print(len(empty_features_test))
    zero_test_features[run] = len(set(empty_features_test) - set(empty_features_train))
    print(zero_test_features[run])
    print()

dataset114k+exp1+exp2_all_lr
(101327, 27551)
(11259, 27551)




218
27551




18163
17955
baseline_randomtest+exp1+exp2_all_lr
(641403, 17744)
(71267, 17744)




8
17744




5683
5675


In [20]:
total_features = {'baseline_randomtest+exp1+exp2_all_lr': 17744,
    'dataset114k+exp1+exp2_all_lr': 27551}

In [21]:
proportions_empty_features = {key: zero_test_features[key]/total_features[key] for key in zero_test_features}
proportions_empty_features

{'dataset114k+exp1+exp2_all_lr': 0.6517004827410984,
 'baseline_randomtest+exp1+exp2_all_lr': 0.3198264201983769}